# Les centres d'intérêts des députés influencent-ils leur participation aux séances en commission permanente ?

## Introduction

Introduction ici: contexte général



présenter la motivation

<b> Stratégie d'idenfication </b>
 présenter la stratégie d'idenfication
 
 <b> Algorithmes </b>
 
 Présenter l'aspect plus algorithmique 
 
 
A demander : 

Récupérer le champ lexical


préférences des députés 



jouer sur le nombre de catégories, voir le threshold de déclenchement 

idée: 

• définir des catégories
•gradient de préférences pour le député
•gradient de thématiques pour la séance
•voir les paires de thématiques qui reviennent le plus souvent les unes à la suite des autres pour en déduire une proximité entre les différentes thématiques 
    - idée probabilité d'avoir un mot à la suite d'un autre 
    
• calculer une distance thématique/préférence, voir le pouvoir prédictif de cette distance


première étape pour les mots: créer une boucle, 1 ligne numero du député, une autre avec toutes ses interventions à la suite (sans traitement) 
filtrer pour les séances en hémicycle uniquement

à partir de ça voir les mots qui sortent le plus souvent pour chacun des députés




 


In [10]:
from jyquickhelper import add_notebook_menu
add_notebook_menu()

## Créer un profil de préférences pour chaque député

Première chose : créer les catégories thématiques (regrouper les champs lexicaux). Agreger toutes les interventions de tous les députés / prendre uniquement les séances en hémicycle et agréger pour tous les députés

A la fin, avoir des mots qui permettent de classifier une thématique de séance
faire du train/test pour voir si ça marche bien


La première chose à faire est de récupérer un fichier contenant toutes les interventions de tous les députés. On importe le fichier <TT>words_mps.txt</TT> qui contient l'ensemble des interventions en hémicycle de l'ensemble des députés de la XIVème législature. Plus précisément, la colonne 0 correspond à l'id du député dans notre base de données (qui nous sera utile plus tard pour retrouver la présence des députés en commission ainsi que leur groupe parlementaire). La colonne 1 est la concaténation brute de toutes ses interventions que l'on va ensuite nettoyer pour extraire les centres d'intérêts de chacun des députés.

Nous utilisons un script autonome pour extraire les données à partir d'une base SQL (cf. annexes)

In [11]:
import pandas 
import numpy as np

#Attention à indiquer l'emplacement local du fichier words_mps.txt (accessible en local uniquement)

#df=pandas.read_csv('/Users/Gabriel/Desktop/words_mps.txt', sep='\t',header=None)
df=pandas.read_csv('/Users/Hugo/Documents/Cours/ENSAE/Python/python_deputes/words_mps.txt', sep='\t',header=None,na_filter=False)
df1 = df.values
df2 = np.array(df)
df.head()

,0,1
0,1,"('<p>Bla-bla-bla !</p>',)(""<p>Vous n'avez enco..."
1,2,"(""<p>Ce n'est pas fini !</p>"",)('<p>Lamentable..."
2,3,"(""<p>J'ai bien entendu la position du Gouverne..."
3,4,"(""<p>C'est vous qui le faites !</p>"",)(""<p>Mon..."
4,5,"(""<p>Monsieur le président, monsieur le minist..."


In [12]:
print(type(df))
print(type(df1))
print(type(df2))
print(df2.shape)
for i in range(0,648) :
    z = df2[i,1]
    z = z.replace("</p>","")
    z = z.replace('"<p>',"")
    z = z.replace("('<p>","")
    z = z.replace("',)(","")
    z = z.replace('",)(','')
    z = z.replace('",)','')
    z = z.replace("',)",'')
    z = z.replace("<p>",'')
    z = z.replace("'"," ")
    z = z.replace("!",'')
    z = z.replace(".",' ')
    df2[i,1] = z
#print(df2)

<class 'pandas.core.frame.DataFrame'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
(648, 2)


Création de la base pour tokenisation avec une seule ligne avec tous les mots employés

In [97]:
import numpy as np

liste=""

for i in range(648):
    liste=liste+df1[i,1]

Nous procédons à la Tokénisation

[Le code pour une tokénisation sans enlever les stops words]
import nltk
#nltk.download('punkt')
from nltk.tokenize import word_tokenize
for i in range(0,648) :
    z = df2[i,1]
    z = ' - '.join(word_tokenize(z))
    df2[i,1] = z
    print(i)
print(df2)

In [13]:
# on enlève les stop-words et on tokénise. En somme on bigdate
import nltk
# nltk.download('stopwords') 
# [Gab il faut que tu télécharges ça, je le mets en commentaire car je l'ai déjà fait]
from nltk.corpus import stopwords
st = set(stopwords.words('french'))
for i in range(0,648) :
    z = df2[i,1]
    z = ' - '.join(w for w in word_tokenize(z) if w not in st)
    df2[i,1] = z
    print(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

Etapes suivantes 

• nettoyer le fichier pour ne garder que les mots clefs
• trouver un moyen de regrouper les mots clef par champ lexical et mettre des catégories dessus (apprentissage non supervisé ?)

In [14]:
print(df2)

[[1
  "Bla-bla-bla - Vous - encore - rien - dit - Vous - déjà - dit - Erreur - N - importe - quoi - Et - Afghanistan - C - pitoyable - C - sûr - De - les - réduire - Il - a - oublié - C - Ayrault - , - zéro - C - redressement - volatile - On - verra - ça - durée - Vous - absolument - répondu - problème - dette - Ben - voyons - Les - Américains - font - Tout - fait - Ce - week-end - encore - Comme - les - 35 - heures - En - effet - Il - a - sans - doute - bien - meilleur - Montebourg - , - homme - redressement - volatil - C - déjà - cas - Ah - non - Allons - , - monsieur - président - Quelle - valeur - ajoutée - voitures - sortent - chaînes - allemandes - ? - Je - voudrais - profiter - défense - cet - amendement - répondre - M - Eckert - , - a - comparé - taux - horaire - les - secteurs - automobiles - allemand - français - Je - prends - acte - chiffres - fournis - , - monsieur - rapporteur - général - On - peut - également - citer - marge - réalisée - les - constructeurs - , - peu - pr

## Attribuer une thématique aux séances en commission

Rebelotte mais pour les séances en commission, très similaire aux députés (cf. script autonome)

In [2]:
import pandas

#Attention à indiquer l'emplacement local du fichier words_meetings.txt (accessible en local uniquement)

#df=pandas.read_csv('/Users/Gabriel/Desktop/words_meetings.txt', sep='\t',header=None)
df=pandas.read_csv('/Users/Hugo/Documents/Cours/ENSAE/Python/python_deputes/words_meetings.txt', sep='\t',header=None)
df.head()

,0,1
0,3,"(""<p>Commission du développement durable et de..."
1,4,"(""<p>COMMISSION DES AFFAIRES CULTURELLES ET DE..."
2,5,"(""<p>La Commission des affaires économiques s'..."
3,6,"(""<p>La Commission a procédé, sous la présiden..."
4,7,"(""<p>COMMISSION DES AFFAIRES SOCIALES</p><p>Je..."


## Voir comment la proximité thématique prédit la participation

## Bibliographie


<span>&#8226;</span> Benjamin Monnery & Maxime Le Bihan, 2018. "Can Public and Private Sanctions Discipline Politicians? Evidence from the French Parliament," EconomiX Working Papers 2018-21, University of Paris Nanterre, EconomiX.

<span>&#8226;</span> Wu, A. H. (2017). Gender stereotyping in academia: Evidence from economics job market rumors forum.
